# 資料載入

In [1]:
import glob
import os
import pandas as pd

fnames = glob.glob("chinese_news_trans/*/*.txt")
datas = {
    "content": [],
    "target": []
}

for fn in fnames:
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    datas["content"].append(content)
    dn = os.path.split(fn)[0]
    dn = os.path.split(dn)[-1]
    datas["target"].append(dn)
train_df = pd.DataFrame(datas)
train_df

,content,target
0,【 日 期 】19960104\n【 版 號 】1\n【 標 題 】合巢蕪高速公路巢蕪...,交通
1,【 日 期 】19960226\n【 版 號 】5\n【 標 題 】我首臺載人磁懸浮列...,交通
2,大秦鐵路萬噸列車試運成功\n新華社北京５月２４日電（通訊員李忠民）大秦...,交通
3,遼寧省檯安縣村村都通柏油路鄉村公路建設全國數第\n一\n (4)...,交通
4,北京—烏蘭巴托—莫斯料３／４次國際旅客列車通車\n３０週年\n ...,交通
...,...,...
2632,馬玉芹破女子４００米跑全國青年紀錄\n新華社鞍山５月１０日電（通訊員王...,體育
2633,國際奧委會中國臺北委員吳經國訪問北京\n新華社北京５月１０日電記者《中...,體育
2634,亞奧理事會３９個成員組織全部以書面形式確認參加\n第十一屆亞運會\n ...,體育
2635,世界盃乒乓球團體賽男子團體採用新賽制\n新華社北京５月１０日電（記者樑...,體育


In [2]:
fnames = glob.glob("chinese_news_test/*/*.txt")
datas = {
    "content": [],
    "target": []
}

for fn in fnames:
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    datas["content"].append(content)
    dn = os.path.split(fn)[0]
    dn = os.path.split(dn)[-1]
    datas["target"].append(dn)
test_df = pd.DataFrame(datas)
test_df

,content,target
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


# targaet 編碼

In [3]:
u = test_df["target"].unique()
trans = {name:i for i, name in enumerate(u)}
reverse_trans = {i:name for name, i in trans.items()}

# print(trans)
# print(reverse_trans)

y_train = train_df["target"].replace(trans)
y_test = test_df["target"].replace(trans)

In [4]:
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: target, Length: 101, dtype: int64

# 辭典載入、資料分詞、資料清洗

In [5]:
# 如果詩詞分類，標點符號和換行，應該去掉，因為它不影響結果。
import jieba
from urllib.request import urlretrieve

# 不存在詞典，就下載
BIG_DICT_PATH = "dict.txt.big"
if not os.path.exists(BIG_DICT_PATH):
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, "dict.txt.big")
else:
    print(BIG_DICT_PATH, "已經存在")
jieba.set_dictionary("dict.txt.big")
    
def news_cut(s):
    s = ' '.join(jieba.cut(s))
    s = s.replace('\r', '').replace('\n', '').replace('\u3000', '').replace('-', '')
    return s

x_train = train_df["content"].apply(news_cut)
x_test = test_df["content"].apply(news_cut)
x_test

Building prefix dict from E:\CLASS\6_Tibame\05_MLDemo_01_v1903\dict.txt.big ...
Loading model from cache C:\Users\his62\AppData\Local\Temp\jieba.u46458f42cd8839b409a3f4ed156be299.cache


dict.txt.big 已經存在


Loading model cost 1.288 seconds.
Prefix dict has been built successfully.


0      日月 光華      Traffic _ Info 精華區 文章 閱讀      發信人 :...
1      日月 光華      Traffic _ Info 精華區 文章 閱讀      發信人 :...
2      日月 光華      Traffic _ Info 精華區 文章 閱讀      發信人 :...
3      三趟 火車 停開   乘客 可 全額 退票    瀏覽 次數 ： 1180         ...
4      日月 光華      Traffic _ Info 精華區 文章 閱讀      發信人 :...
                             ...                        
96     最 優秀 選手 無緣 亞運會 健美 賽                 健美 在 亞洲 運動...
97        各國 記者 眼中 的 羽毛球 世錦賽           2001  06  13  ...
98      友好 運動會 第五天   東道主 選手 大顯神威  2001 年 09 月 03 日 02...
99     不靠 技術 比 運氣     第二屆 奧運會 在 巴黎 舉行 ， 同時 這裡 也 正在 舉行...
100    帆   板   運   動   簡   介 （ 二 ）     我國 在 79 年 由 國家...
Name: content, Length: 101, dtype: object

# 特徵提取

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
x_train_vec = vec.fit_transform(x_train)
x_test_vec  = vec.transform(x_test)

# 資料訓練

In [7]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB() # 預設已經有做 laplace smoothing
clf.fit(x_train_vec, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [8]:
from sklearn.metrics import accuracy_score

pre = clf.predict(x_test_vec)
print("正確率: ", accuracy_score(pre, y_test))

正確率:  1.0


In [9]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_test, pre)
c = [f"{name}(預測)" for name in u]
i = [f"{name}(原本)" for name in u]
pd.DataFrame(mat, columns=c, index=i)

,交通(預測),政治(預測),教育(預測),環境(預測),經濟(預測),藝術(預測),計算機(預測),軍事(預測),醫藥(預測),體育(預測)
交通(原本),10,0,0,0,0,0,0,0,0,0
政治(原本),0,11,0,0,0,0,0,0,0,0
教育(原本),0,0,10,0,0,0,0,0,0,0
環境(原本),0,0,0,10,0,0,0,0,0,0
經濟(原本),0,0,0,0,10,0,0,0,0,0
藝術(原本),0,0,0,0,0,10,0,0,0,0
計算機(原本),0,0,0,0,0,0,10,0,0,0
軍事(原本),0,0,0,0,0,0,0,10,0,0
醫藥(原本),0,0,0,0,0,0,0,0,10,0
體育(原本),0,0,0,0,0,0,0,0,0,10


# 模型演練
[yahoo news](https://tw.news.yahoo.com/)

In [15]:
p = input("輸入一篇新聞: ")
p = vec.transform([news_cut(p)])

ans = clf.predict(p)[0]
proba = clf.predict_proba(p)[0]

print("分類應該是: ", reverse_trans[ans])

percent = sorted(zip(u, proba), key=lambda x: x[1], reverse=True)
for n, prob in percent:
    print(n, "\t的機率:\t", prob * 100, '%')

輸入一篇新聞:  台灣連續4天0確診，先前國人很擔心磐石艦隊確診官兵下船後，和女友親密接觸恐成防疫破口，不過經過篩檢，海軍確診者的女友都沒有人確診，也讓外界直呼「很神」，對此，中央流行疫情指揮中心陳時中今（29）日說明關鍵，他認為主要原因，可能是磐石軍艦上的確診者相對年輕、且都是輕症患者，但還要持續觀察。  磐石艦爆發武漢肺炎群聚感染，目前共31人確診，根據指揮中心疫調指出，有13名染疫官兵曾與女友有親密接觸，不過，經過篩檢後，至今都沒有傳出確診者接觸者確診，也讓人不免好奇「喝咖啡都會染疫了，親密接觸居然不會？」  對此，陳時中今天表示，親密接觸者沒有被檢驗出陽性，可能是跟敦睦艦隊確診者相對年輕有關，陳時中也指出，艦隊裡面輕症患者比較多，但還是持續觀察。  台灣目前共429例確診，分別為343例境外移入，55例本土病例及31例敦睦艦隊。確診個案中6人死亡，311人解除隔離，其餘持續住院隔離中。


分類應該是:  軍事
軍事 	的機率:	 99.99999999899956 %
醫藥 	的機率:	 9.951581765149984e-10 %
經濟 	的機率:	 2.8061129446189386e-12 %
環境 	的機率:	 4.634741041880521e-18 %
計算機 	的機率:	 3.2263310621737477e-18 %
藝術 	的機率:	 4.41018674696302e-22 %
體育 	的機率:	 2.7243046337594694e-22 %
交通 	的機率:	 1.048312688959263e-23 %
教育 	的機率:	 3.642765859774093e-26 %
政治 	的機率:	 1.8858420446445501e-26 %
